In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor

%matplotlib inline

In [2]:
!ls Data

reviews_Beauty_5.json.gz


In [3]:
df = pd.read_json("Data/reviews_Beauty_5.json.gz",lines=True)

In [4]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [5]:
df.isna().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [6]:
#1386 reviewerNames left blank; we will not need reviewerName since utilizing reviewerID but 
#curious why

In [7]:
df[df['reviewerName'].isnull()]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
8,A3LMILRM9OC3SA,9759091062,NaN,"[0, 0]",Did nothing for me. Stings when I put it on. I...,2,"no Lightening, no Brightening,......NOTHING",1405209600,"07 13, 2014"
1790,AK1H26O8DLMNN,B0000535UM,NaN,"[0, 0]",The first thickening shampoo that works on my ...,5,Actually works,1405123200,"07 12, 2014"
2242,APTLHR9PHGPXN,B00005NAOD,NaN,"[0, 0]","Kind of drying, not moisturizing. Kind of disa...",2,Inexpensive and feels that wY,1405209600,"07 13, 2014"
2304,AQWX644AFUFFK,B00005NFBD,NaN,"[0, 0]","This is just ok. For one, I found this in a st...",3,"Ok, nothing amazing",1405468800,"07 16, 2014"
3651,A43K5ZRQ87TO6,B00008PC1O,NaN,"[0, 0]",Works well and easy to use!,5,Five Stars,1405296000,"07 14, 2014"
...,...,...,...,...,...,...,...,...,...
197192,A1Z3AV93ONK5VF,B00KAL5JAU,NaN,"[0, 0]",We already had the Dead Sea Shampoo by Adovia ...,5,"Non greasy, silky shiny hair",1401235200,"05 28, 2014"
197193,A184I8GT3BHZQV,B00KAL5JAU,NaN,"[0, 1]",&#60;a href=&#34;http://www.tomoson.com/?code=...,5,Dead Sea Salt hair conditioner,1403568000,"06 24, 2014"
197194,A8C9EJORQD23,B00KAL5JAU,NaN,"[0, 1]",I use this with the Adovia shampoo I mention a...,5,Adovia does natural right!,1402272000,"06 9, 2014"
198446,A2PIGZCDGM4NJ7,B00L5JHZJO,NaN,"[10, 11]","This is a horrible product, most of the review...",1,Bad Product,1404864000,"07 9, 2014"


In [8]:
#all necessary information is included even when reviewerName is NaN

In [9]:
df.reviewerID.value_counts()

A2V5R832QCSOMX    204
ALNFHVS3SC4FV     192
AKMEY1BSHSDG7     182
A3KEZLJ59C1JVH    154
ALQGOMOY1F5X9     150
                 ... 
AFAIEJL65GKRK       5
A1DXDPN8720R2U      5
A38MNWW6OKA6KI      5
A5XF76PK1PEYO       5
A35K5RQB1J9LP       5
Name: reviewerID, Length: 22363, dtype: int64

In [10]:
#YAY! all reviewerIDs have value of at least 5, total of 22,363 reviewers

In [11]:
df.asin.value_counts()

B004OHQR1Q    431
B0043OYFKU    403
B0069FDR96    391
B000ZMBSPE    389
B00150LT40    329
             ... 
B0033I9Y1C      5
B0001ZZH0M      5
B004LXKY4E      5
B003JCLP6E      5
B000UPEF88      5
Name: asin, Length: 12101, dtype: int64

In [12]:
#YAY! all products have at least 5 reviews, total of 12,101 different products

In [13]:
#can save data as a CSV file for easier use, but it does not look like we need to clean data
#can drop reviewerNames

In [14]:
lower_rating = df.overall.min()

In [15]:
upper_rating = df.overall.max()

In [16]:
print('Review range: {0} to {1}'.format(lower_rating, upper_rating))

Review range: 1 to 5


In [17]:
#Confirming our review range is 1 to 5, which is the default for surprise

In [18]:
#Creating dataframe with appropriate columns to run through surprise

In [19]:
surprise_df = df[['reviewerID', 'asin', 'overall']]

In [20]:
surprise_df

,reviewerID,asin,overall
0,A1YJEY40YUW4SE,7806397051,1
1,A60XNB876KYML,7806397051,3
2,A3G6XNM240RMWA,7806397051,4
3,A1PQFP6SAJ6D80,7806397051,2
4,A38FVHZTNQ271F,7806397051,3
...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,5
198498,A1UQBFCERIP7VJ,B00LLPT4HI,5
198499,A35Q0RBM3YNQNF,B00LLPT4HI,5
198500,A3LGT6UZL99IW1,B00LLPT4HI,5


In [21]:
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(surprise_df, reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=23)

In [22]:
surprise_data

In [23]:
# How many users and items are in the trainset
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  22359 

Number of items:  12101 



In [24]:
#10,000 more users than items

In [25]:
print('Type trainset :',type(trainset),'\n')
print('Type testset :',type(testset))

Type trainset : <class 'surprise.trainset.Trainset'> 

Type testset : <class 'list'>


In [26]:
print(len(testset))
print(testset[0])

39701
('A32POM4ALTYIZV', 'B005Z4QT7E', 2.0)


In [27]:
print(len(df))

198502


In [28]:
198502 * .2

39700.4

In [29]:
#sanity check that test set is 20% of total data

In [30]:
#starting with item-item similarity

In [31]:
sim_cos = {'name':'cosine', 'user_based':False}

In [32]:
from surprise.prediction_algorithms import knns

In [33]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [34]:
# looking at the similarity metrics of each of the items to one another by using the sim attribute of our fitted model

In [35]:
basic.sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [36]:
predictions = basic.test(testset)

In [37]:
print(accuracy.rmse(predictions))

RMSE: 1.2129
1.2128933192100995


In [38]:
#RMSE of about 1.2, meaning that it was off by roughly 1.2 points for each guess it made for ratings

In [39]:
sim_cos_user = {'name':'cosine', 'user_based':True}

In [40]:
basic_user = knns.KNNBasic(sim_options=sim_cos_user)
basic_user.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [41]:
basic_user.sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [42]:
predictions_user = basic_user.test(testset)

In [43]:
print(accuracy.rmse(predictions_user))

RMSE: 1.2401
1.2400588428109054


In [44]:
#KNN model with Pearson Correlation 

In [45]:
sim_pearson = {'name':'pearson', 'user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1854
1.1854039491870065


In [46]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
basic_pearson_user = knns.KNNBasic(sim_options=sim_pearson_user)
basic_pearson_user.fit(trainset)
predictions_user = basic_pearson_user.test(testset)
print(accuracy.rmse(predictions_user))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1795
1.1795381227887218


In [47]:
#KNN model with Means

In [48]:
#same thing as the basic KNN model, except it takes into account the mean rating of each user or item depending on 
#whether you are performing user-user or item-item similarities, respectively.

In [49]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1791
1.1790736593254243


In [50]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
knn_means_user = knns.KNNWithMeans(sim_options=sim_pearson_user)
knn_means_user.fit(trainset)
predictions_user = knn_means_user.test(testset)
print(accuracy.rmse(predictions_user))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1622
1.162184545973618


In [51]:
#This is our best model so far

In [52]:
#Running an SVD model with defaults

In [53]:
svd = SVD(random_state=42)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.0889
1.0889451149217502


In [54]:
#Lowest RMSE so far!

In [55]:
svd.predict('A1YJEY40YUW4SE', 'B00LLPT4HI')

Prediction(uid='A1YJEY40YUW4SE', iid='B00LLPT4HI', r_ui=None, est=4.41104541567184, details={'was_impossible': False})

In [56]:
svd.predict('A2BLFCOPSMBOZ9', '7806397051')

Prediction(uid='A2BLFCOPSMBOZ9', iid='7806397051', r_ui=None, est=3.774499861592997, details={'was_impossible': False})

In [57]:
#Checking to see estimated rating for 2 user/product combinations

In [58]:
trainset

In [59]:
cv_svd_baseline = cross_validate(svd, surprise_data)

In [60]:
cv_svd_baseline

{'test_rmse': array([1.08980963, 1.08591658, 1.09065492, 1.08666649, 1.09817742]),
 'test_mae': array([0.83597309, 0.83313975, 0.83726676, 0.83471144, 0.83813063]),
 'fit_time': (8.994711875915527,
  9.048747062683105,
  9.145930051803589,
  9.050917863845825,
  9.20858097076416),
 'test_time': (0.2082078456878662,
  0.2016451358795166,
  0.20255112648010254,
  0.2091081142425537,
  0.4840390682220459)}

In [61]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [62]:
#Baseline Models; true baseline should be NormalPredictor(); user_based=True

In [63]:
baseline = NormalPredictor()
baseline.fit(trainset)

In [64]:
predictions = baseline.test(testset)

In [65]:
baseline = accuracy.rmse(predictions)

RMSE: 1.4998


In [66]:
baseline2 = BaselineOnly()
baseline2.fit(trainset)

Estimating biases using als...


In [67]:
predictions2 = baseline2.test(testset)

In [68]:
baseline2 = accuracy.rmse(predictions2)

RMSE: 1.0890
